In [4]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm.notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Load files into a pandas dataframe
train = pd.read_csv('Train (2).csv',parse_dates=['Policy Start Date','Policy End Date','First Transaction Date'],index_col='ID')
test = pd.read_csv('Test (1).csv',parse_dates=['Policy Start Date','Policy End Date','First Transaction Date'],index_col='ID')
ss = pd.read_csv('SampleSubmission (1).csv')

In [6]:
y=train['target']

In [7]:
train = train.drop('target',axis=1)

In [8]:
train.head()

,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName
ID,,,,,,,,,,,,
ID_0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic
ID_0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic
ID_005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic
ID_0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe
ID_00BRP63,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve


In [9]:
train.shape

(12079, 12)

In [10]:
test.shape

(5177, 12)

In [11]:
train.isnull().sum()/len(train)* 100

Policy Start Date          0.000000
Policy End Date            0.000000
Gender                     2.972100
Age                        0.000000
First Transaction Date     0.000000
No_Pol                     0.000000
Car_Category              30.946270
Subject_Car_Colour        57.637222
Subject_Car_Make          20.498386
LGA_Name                  53.613710
State                     53.713056
ProductName                0.000000
dtype: float64

In [12]:
test.isnull().sum()/len(test)*100

Policy Start Date          0.000000
Policy End Date            0.000000
Gender                     3.013328
Age                        0.000000
First Transaction Date     0.000000
No_Pol                     0.000000
Car_Category              31.639946
Subject_Car_Colour        58.045200
Subject_Car_Make          20.494495
LGA_Name                  53.737686
State                     53.853583
ProductName                0.000000
dtype: float64

In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12079 entries, ID_0040R73 to ID_ZZWRIIE
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Policy Start Date       12079 non-null  datetime64[ns]
 1   Policy End Date         12079 non-null  datetime64[ns]
 2   Gender                  11720 non-null  object        
 3   Age                     12079 non-null  int64         
 4   First Transaction Date  12079 non-null  datetime64[ns]
 5   No_Pol                  12079 non-null  int64         
 6   Car_Category            8341 non-null   object        
 7   Subject_Car_Colour      5117 non-null   object        
 8   Subject_Car_Make        9603 non-null   object        
 9   LGA_Name                5603 non-null   object        
 10  State                   5591 non-null   object        
 11  ProductName             12079 non-null  object        
dtypes: datetime64[ns](3), int64(2), objec

In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5177 entries, ID_009D84L to ID_ZZYTLV1
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Policy Start Date       5177 non-null   datetime64[ns]
 1   Policy End Date         5177 non-null   datetime64[ns]
 2   Gender                  5021 non-null   object        
 3   Age                     5177 non-null   int64         
 4   First Transaction Date  5177 non-null   datetime64[ns]
 5   No_Pol                  5177 non-null   int64         
 6   Car_Category            3539 non-null   object        
 7   Subject_Car_Colour      2172 non-null   object        
 8   Subject_Car_Make        4116 non-null   object        
 9   LGA_Name                2395 non-null   object        
 10  State                   2389 non-null   object        
 11  ProductName             5177 non-null   object        
dtypes: datetime64[ns](3), int64(2), object

In [15]:
train.Car_Category.fillna('Others',inplace =True)
train['Car_Category'] = train['Car_Category'].replace({"Mini Van": "Van",
                                                   "Pick Up > 3 Tons": "Pick Up","Mini Bus": "Bus","Sedan": "Saloon"
                                                    ,"CAMRY CAR HIRE": "Saloon","Tipper Truck": "Truck"
                                                    ,"Shape Of Vehicle Chasis": "Others","Station 4 Wheel": "Others"})
test.Car_Category.fillna('Others',inplace =True)
test['Car_Category'] = test['Car_Category'].replace({"Mini Van": "Van",
                                                   "Pick Up > 3 Tons": "Pick Up","Mini Bus": "Bus","Sedan": "Saloon"
                                                    ,"CAMRY CAR HIRE": "Saloon","Tipper Truck": "Truck"
                                                    ,"Shape Of Vehicle Chasis": "Others","Station 4 Wheel": "Others"})

In [16]:
train.Car_Category.unique()

array(['Saloon', 'JEEP', 'Others', 'Motorcycle', 'Truck', 'Bus',
       'Pick Up', 'Van', 'Wagon'], dtype=object)

In [17]:
train.LGA_Name.unique()

array([nan, 'Lagos', 'Ikeja', 'Badagry', 'Eti-Osa', 'Victoria Island',
       'Ikoyi', 'Lagos Mainland', 'EFFURUN', 'Abeokuta',
       'Abuja Municipal', 'Yaba', 'Aguda', 'Surulere', 'Oshodi-Isolo',
       'Alimosho', 'Okpe, Delta State', 'Asaba', 'IbadanCentral', 'Lekki',
       'Ibeju/Lekki', 'OBANIKORO', 'Agege', 'ISHERI', 'Kosofe', 'OGUN',
       'Apapa', 'Ibadan South West', 'Obia/Akpor', 'Ibadan South East',
       'Sapele', 'Kaduna South', 'Ikorodu', 'Amuwo-Odofin', 'Awka',
       'GBAGADA', 'Port Harcourt', 'Minna', 'Shomolu', 'Benin',
       'Lagos Island', 'Central', 'Ebute-Metta', 'Ketu', 'Uyo', 'Mushin',
       'Katagum', 'ENUGU EAST', 'Oredo', 'Abuja', 'Festac', 'Ipaja',
       'Zaria ', 'Esan West', 'Oluyole', 'Bwari', 'Obafemi-Owode', 'LGA',
       'Orile-Iganmu', 'Egbeda', 'Ido', 'Ifako-Ijaye ', 'Ibadan North',
       'Oguta', 'Jos North', 'Ajah', 'Ondo West', 'Kaduna North', 'Warri',
       'Aniocha South', 'Owerri', 'Awka South', 'Asokoro District',
       'Ile-Oluji'

In [18]:
train['LGA_Name']=train.LGA_Name.replace({'LGA':'Unknown'})
test['LGA_Name']=test.LGA_Name.replace({'LGA':'Unknown'})
train.LGA_Name.fillna('Unknown',inplace =True)
test.LGA_Name.fillna('Unknown',inplace=True)

In [19]:
train.Subject_Car_Colour.unique()

array(['Black', 'Grey', 'Red', nan, 'As Attached', 'Blue', 'Silver',
       'Green', 'Ash', 'White', 'Gray & Gray', 'Wine', 'Brown',
       'Red & Blue', 'Gold', 'Gray & Silver', 'Orange', 'Red & Black',
       'Dark Grey', 'White & Red', 'Light Green', 'Dark Gray', 'B.Silver',
       'Purple', 'Blue & Red', 'Red & Yellow', 'Yellow', 'Red & White',
       'D. Red', 'Dark Red', 'Black & White', 'White & Blue', 'Cream',
       'Dark Green', 'D. Gold', 'Champagne', 'Light Blue',
       'Blue&White&Red', 'White & Yellow', 'Dark Blue', 'Black & Orange',
       'Burgundy', 'Yellow & White', 'Beige Mitalic', 'Light Gray',
       'Blue Sky'], dtype=object)

In [20]:
train.Subject_Car_Colour.fillna('Unknown',inplace =True)
train['Subject_Car_Colour'] = train['Subject_Car_Colour'].replace({"White & Blue": "Multicolor",
                                                   "Red & White": "Multicolor","Blue & Red": "Multicolor","Gray & Gray": "Grey"
                                                    ,"Black & Orange": "Multicolor","Red & Black": "Multicolor","Gray & Silver": "Multicolor"
                                                    ,"Yellow & White": "Multicolor","Red & Yellow": "Multicolor"
                                                              ,"Blue&White&Red": "Multicolor" ,"Black & White": "Multicolor" 
                                                             ,"White & Red": "Multicolor","Red & Blue": "Multicolor" , "White & Yellow": "Multicolor"
                                                            ,'D. Red':'Dark Red','D. Gold':'Champagne','Dark Grey':'Dark Gray','Blue Sky':'Light Blue'})

test.Subject_Car_Colour.fillna('Unknown',inplace =True)
test['Subject_Car_Colour'] = test['Subject_Car_Colour'].replace({"White & Blue": "Multicolor",
                                                   "Red & White": "Multicolor","Blue & Red": "Multicolor","Gray & Gray": "Grey"
                                                    ,"Black & Orange": "Multicolor","Red & Black": "Multicolor","Gray & Silver": "Multicolor"
                                                    ,"Yellow & White": "Multicolor","Red & Yellow": "Multicolor"
                                                              ,"Blue&White&Red": "Multicolor" ,"Black & White": "Multicolor" 
                                                             ,"White & Red": "Multicolor","Red & Blue": "Multicolor" , "White & Yellow": "Multicolor"
                                                                ,'D. Red':'Dark Red','D. Gold':'Champagne','Dark Grey':'Dark Gray','Blue Sky':'Light Blue'})

In [21]:
train.Subject_Car_Colour.unique()

array(['Black', 'Grey', 'Red', 'Unknown', 'As Attached', 'Blue', 'Silver',
       'Green', 'Ash', 'White', 'Wine', 'Brown', 'Multicolor', 'Gold',
       'Orange', 'Dark Gray', 'Light Green', 'B.Silver', 'Purple',
       'Yellow', 'Dark Red', 'Cream', 'Dark Green', 'Champagne',
       'Light Blue', 'Dark Blue', 'Burgundy', 'Beige Mitalic',
       'Light Gray'], dtype=object)

In [22]:
train.Gender.unique()

array(['Male', 'Female', 'Entity', 'Joint Gender', nan, 'NO GENDER',
       'NOT STATED', 'SEX'], dtype=object)

In [23]:
train['Gender'] =train['Gender'].replace({"NO GENDER": "NOT STATED",
                                                   "SEX": "NOT STATED","NO GENDER": "NOT STATED",'Joint Gender':'NOT STATED','Entity':'NOT STATED'})
train['Gender'].fillna('NOT STATED', inplace=True)



test['Gender'] =test['Gender'].replace({"NO GENDER": "NOT STATED",
                                                   "SEX": "NOT STATED","NO GENDER": "NOT STATED",'Joint Gender':'NOT STATED','Entity':'NOT STATED'})
test['Gender'].fillna('NOT STATED', inplace=True)

In [24]:
date_columns = ['Policy Start Date','Policy End Date','First Transaction Date']


In [25]:
def extract_date_info(df,cols,):
    for feat in cols:
        df[feat +'_year'] = df[feat].dt.quarter
        df[feat +'_day'] = df[feat].dt.day
        df[feat +'_month'] = df[feat].dt.month
        df[feat +'_quarter'] = df[feat].dt.quarter
    df.drop(columns=date_columns,axis=1,inplace=True)

In [26]:
extract_date_info(train,date_columns)
extract_date_info(test,date_columns)

In [27]:
train.State.unique()

array([nan, 'Lagos', 'Benue', 'Eti-Osa', 'Delta', 'Ogun',
       'Abuja-Municipal', 'Oshodi-Isolo', 'Ibeju-Lekki', 'Ibadan-West',
       'Obia-Akpor', 'Ibadan-East', 'Kaduna-South', 'Amuwo-Odofin',
       'Anambra', 'Rivers', 'Niger-State', 'Edo', 'Akwa-Ibom',
       'ENUGU-EAST', 'Abuja', 'Kaduna', 'Esan-West', 'Obafemi-Owode',
       'Orile-Iganmu', 'Ifako-Ijaye', 'Ibadan-North', 'Imo', 'Jos-North',
       'Ondo-West', 'Kaduna-North', 'Aniocha-South', 'Awka-South',
       'Ile-Oluji', 'Ijebu-Ode', 'Port-Harcourt', 'Nnewi-North',
       'Aboh-Mbaise', 'Oyo', 'Akoko-West', 'Warri-Central', 'Cross-River',
       'Jos-South', 'Onitsha-North', 'Osun', 'Ajeromi-Ifelodun',
       'Kano-Municipal', 'Ife-Central', 'Ilorin-West', 'Lagelu-North',
       'Owerri-Municipal', 'Nsit-Ubium', 'Udi-Agwu', 'Essien-Udim',
       'Owerri-West', 'Ondo', 'Ogbmosho-South', 'Umuahia-South',
       'ABULE-EGBA', 'Enugu-North', 'Ovia-SouthWest', 'Kebbi',
       'Ajegunle-State', 'Ekiti-West', 'Ogun-Waterside',

In [28]:
train['State'] = train['State'].replace({'Abuja-Municipal':'Abuja','Oshodi-Isolo':'Lagos','Ibeju-Lekki':'Lagos','Ibadan-West':'Ibadan','Eti-Osa':'Lagos'
                                        ,'Obia-Akpor':'Awka-Ibom','Ibadan-East':'Oyo','Amuwo-Odofin':'Lagos','ENUGU-EAST':'Enugu','Esan-West':'Edo'
                                         ,'Obafemi-Owode':'Ogun','Orile-Iganmu':'Lagos', 'Ifako-Ijaye':'Lagos', 'Ibadan-North':'Oyo',
                                         'Jos-North':'Plateau'
                                        , 'Ondo-West':'Ondo', 'Kaduna-North':'Kaduna','Aniocha-South':'Delta','Ile-Oluji':'Ondo','Nnewi-North':'Anambra'
                                        ,'Aboh-Mbaise':'Imo', 'Akoko-West':'Ondo','Warri-Central':'Edo', 'Jos-South':'Plateus','Onitsha-North':'Anambra'
                                        , 'Ajeromi-Ifelodun':'Osun', 'Kano-Municipal':'Kano','Ife-Central':'Osun','Ilorin-West':'Kwara'
                                         ,'Lagelu-North':'Oyo', 'Owerri-Municipal':'Imo','Nsit-Ubium':'Awka-Ibom', 'Udi-Agwu':'Enugu',
                                         'Owerri-West':'Imo'
                                         ,'Ogbmosho-South':'Oyo', 'Umuahia-South':'Abia','Ajegunle-State':'Lagos','Ekiti-West':'Ekiti',
                                         'Ogun-Waterside':'Ogun'
                                        ,'Awka-North':'Anambra','ABULE-EGBA':'Lagos','Ife-North':'Osun', 'Anambra-East':'Anambra', 'Etsako-West':'Edo'
                                         ,'Aba-North':'Abia','Ibarapa-Central':'Oyo', 'Idemili-North':'Anambra', 'Ogba-Ndoni':'Rivers', 
                                         'Calabar-Municipality':'Cross-River','AJAO-ESTATE':'Lagos','Ekiti-East':'Ekiti',
                                         'Ovia-SouthWest':'Edo','Essien-Udim':'Akwa-Ibom',
                                         'Ado-Ota':'Ogun', 'Oyo-East':'Oyo', 'Ilorin-East':'Kwara', 'Ilesha-West':'Osun', 'Central-Abuja':'Abuja',
                                         'Owerri-North':'Imo','Ilesha-East':'Osun', 'Awka-South':'Delta','Enugu-North':'Enugu','Ibadan':'Oyo',
                                         'Esan-Central':'Edo', 'Isoko-south':'Delta', 'Ethiope-East':'Delta','Ijebu-Ode':'Ogun'
                                         ,'QuaAn-Pan':'Plateus', 'Warri-North':'Delta', 'Ado-Ekiti':'Ekiti', 'Ijebu-East':'Ogun',
                                         'Onitsha-South':'Anambra', 'Ughelli-North':'Delta', 'Warri-South':'Delta', 'Kaduna-South':'Kaduna',
                                         'Port-Harcourt':'Rivers',
      
                                         'Oyo-West':'Oyo', 'Aba-South':'Abia', 'Isoko-North':'Delta', 'Oshimili-North':'Delta',
                                         'Ndokwa-East':'Delta', 'Nnewi-South':'Anambra', 'ENUGU-SOUTH':'Enugu', 'Ijebu-North':'Ogun',
                                         'Asari-Toru':'Rivers'})

In [29]:
train.State.fillna('N-A',inplace =True)
test.State.fillna('N-A',inplace =True)

In [30]:
test.State.unique()

array(['Lagos', 'Abuja-Municipal', 'Benue', 'Abuja', 'N-A', 'Eti-Osa',
       'Obia-Akpor', 'Amuwo-Odofin', 'Jos-North', 'Kwara', 'Oyo',
       'Ijebu-Ode', 'Rivers', 'Edo', 'Oshodi-Isolo', 'Osun',
       'Kaduna-North', 'Ibarapa-Central', 'Warri-Central', 'Delta',
       'Kaduna', 'Ado-Ota', 'Ibadan-West', 'Lagelu-North', 'Ifako-Ijaye',
       'Ibeju-Lekki', 'Anambra', 'Ibadan-East', 'Enugu-North',
       'ENUGU-EAST', 'Kogi', 'Ogun', 'Jos-South', 'Ondo', 'Ife-North',
       'Akwa-Ibom', 'Ogbmosho-South', 'Warri-South', 'Anambra-East',
       'Port-Harcourt', 'Kano-Municipal', 'Owerri-Municipal',
       'Central-Abuja', 'Ile-Oluji', 'Ibadan-North', 'Imo',
       'Ughelli-North', 'AJAO-ESTATE', 'Ndokwa-East', 'Onitsha-South',
       'Ilorin-West', 'Ijebu-East', 'Aba-North', 'Aniocha-South',
       'Nasarawa', 'Aboh-Mbaise', 'Gombe', 'Ife-Central', 'Abia',
       'Niger-State', 'Ethiope-East', 'Ogun-Waterside', 'Cross-River',
       'Kaduna-South', 'Owerri-North', 'Esan-West', 'Nnewi-No

In [31]:
test['State'] = test['State'].replace({'Abuja-Municipal':'Abuja','Oshodi-Isolo':'Lagos','Ibeju-Lekki':'Lagos','Ibadan-West':'Ibadan','Eti-Osa':'Lagos'
                                        ,'Obia-Akpor':'Awka-Ibom','Ibadan-East':'Oyo','Amuwo-Odofin':'Lagos','ENUGU-EAST':'Enugu','Esan-West':'Edo'
                                         ,'Obafemi-Owode':'Ogun','Orile-Iganmu':'Lagos', 'Ifako-Ijaye':'Lagos', 'Ibadan-North':'Oyo',
                                         'Jos-North':'Plateau'
                                        , 'Ondo-West':'Ondo', 'Kaduna-North':'Kaduna','Aniocha-South':'Delta','Ile-Oluji':'Ondo','Nnewi-North':'Anambra'
                                        ,'Aboh-Mbaise':'Imo', 'Akoko-West':'Ondo','Warri-Central':'Edo', 'Jos-South':'Plateus','Onitsha-North':'Anambra'
                                        , 'Ajeromi-Ifelodun':'Osun', 'Kano-Municipal':'Kano','Ife-Central':'Osun','Ilorin-West':'Kwara'
                                         ,'Lagelu-North':'Oyo', 'Owerri-Municipal':'Imo','Nsit-Ubium':'Awka-Ibom', 'Udi-Agwu':'Enugu',
                                         'Owerri-West':'Imo'
                                         ,'Ogbmosho-South':'Oyo', 'Umuahia-South':'Abia','Ajegunle-State':'Lagos','Ekiti-West':'Ekiti',
                                         'Ogun-Waterside':'Ogun'
                                        ,'Awka-North':'Anambra','ABULE-EGBA':'Lagos','Ife-North':'Osun', 'Anambra-East':'Anambra', 'Etsako-West':'Edo'
                                         ,'Aba-North':'Abia','Ibarapa-Central':'Oyo', 'Idemili-North':'Anambra', 'Ogba-Ndoni':'Rivers', 
                                         'Calabar-Municipality':'Cross-River','AJAO-ESTATE':'Lagos','Ekiti-East':'Ekiti',
                                         'Ovia-SouthWest':'Edo','Essien-Udim':'Awka-Ibom',
                                         'Ado-Ota':'Ogun', 'Oyo-East':'Oyo', 'Ilorin-East':'Kwara', 'Ilesha-West':'Osun', 'Central-Abuja':'Abuja',
                                         'Owerri-North':'Imo','Ilesha-East':'Osun', 'Awka-South':'Delta','Enugu-North':'Enugu','Ibadan':'Oyo',
                                         'Esan-Central':'Edo', 'Isoko-south':'Delta', 'Ethiope-East':'Delta','Ijebu-Ode':'Ogun'
                                         ,'QuaAn-Pan':'Plateus', 'Warri-North':'Delta', 'Ado-Ekiti':'Ekiti', 'Ijebu-East':'Ogun',
                                         'Onitsha-South':'Anambra', 'Ughelli-North':'Delta', 'Warri-South':'Delta', 'Kaduna-South':'Kaduna',
                                         'Port-Harcourt':'Rivers',
      
                                         'Oyo-West':'Oyo', 'Aba-South':'Abia', 'Isoko-North':'Delta', 'Oshimili-North':'Delta',
                                         'Ndokwa-East':'Delta', 'Nnewi-South':'Anambra', 'ENUGU-SOUTH':'Enugu', 'Ijebu-North':'Ogun',
                                         'Asari-Toru':'Rivers','Ngor-Okpala':'Imo','Idemili-Sorth':'Anambra'})

In [32]:
train.Subject_Car_Make.unique()

array(['TOYOTA', nan, 'REXTON', 'Lexus', 'Hyundai', 'Iveco', 'DAF',
       'Honda', 'Mercedes', 'Jincheng', 'ACURA', 'Ford', 'Volkswagen',
       'Nissan', '.', 'Pontiac', 'Range Rover', 'Kia', 'Mitsubishi',
       'Scania', 'BMW', 'Infiniti', 'Renault', 'Volvo', 'Hummer', 'Mack',
       'Grand Cherokee', 'Porsche', 'Peugeot', 'Land Rover', 'Chevrolet',
       'Mazda', 'Man', 'Jeep', 'Audi', 'Suzuki', 'MINI COOPER', 'KA',
       'As Attached', 'Innson', 'Isuzu', 'Skoda', 'Jaguar', 'Chrysler',
       'Dodge', 'GMC', 'Land Rover.', 'Subaru', 'GAC', 'Fiat', 'Opel',
       'Astra', 'Motorcycle', 'ZOYTE', 'Seat', 'FOTON', 'Howo', 'Lincoln',
       'Rols Royce', 'CHANGAN', 'LIBERTY', 'ABG', 'Yamaha', 'Black',
       'Tata', 'Ashok Leyland', 'Geely', 'Caddillac', 'Wrangler Jeep',
       'Raston', 'MG', 'BRILLIANCE', 'Buik', 'COMMANDER', 'Bajaj'],
      dtype=object)

In [33]:
 train['Subject_Car_Make']=train.Subject_Car_Make.replace({'ACURA':'Honda','Infiniti':'Nissan', 'Grand Cherokee':'Jeep',
                                                           'Volkswagen':'Audi','KA':'Ford','Innson':'Innoson','Land Rover':'Jaguar',
                                                          'Volkswagen':'Skoda','Land Rover.':'Jaguar','Fiat':'Chrysler',
                                                           'Astra':'Opel'
                                                          ,'Seat':'Volkswagen','Lincoln':'Ford','LIBERTY':'Jeep','Wrangler Jeep':'Jeep'
                                                           ,'Buik':'GMC','COMMANDER':'Jeep','Datsun':'Nissan','.':'Unknown'})
train.Subject_Car_Make.fillna('Unknown',inplace =True)

In [34]:
test.Subject_Car_Make.unique()

array(['Iveco', 'TOYOTA', 'Ford', nan, 'Honda', 'Hyundai', 'ACURA',
       'Volkswagen', 'Nissan', 'Range Rover', 'Lexus', 'Mercedes', 'Kia',
       'Chevrolet', '.', 'Skoda', 'Infiniti', 'Renault', 'Mazda',
       'Mitsubishi', 'Suzuki', 'Peugeot', 'Mack', 'GMC', 'Land Rover',
       'Dodge', 'As Attached', 'Chrysler', 'Jeep', 'Pontiac', 'BMW',
       'Subaru', 'Volvo', 'MINI COOPER', 'Opel', 'DAF', 'MG',
       'Wrangler Jeep', 'Audi', 'Innson', 'Scania', 'Porsche', 'LIBERTY',
       'KA', 'Black', 'Datsun', 'Land Rover.', 'Isuzu', 'Hummer',
       'Grand Cherokee', 'FOTON', 'Buik', 'Seat', 'Man'], dtype=object)

In [39]:
test['Subject_Car_Make']=test.Subject_Car_Make.replace({'ACURA':'Honda','Infiniti':'Nissan', 'Grand Cherokee':'Jeep',
                                                           'Volkswagen':'Audi','KA':'Ford','Innson':'Innoson','Land Rover':'Jaguar',
                                                          'Volkswagen':'Skoda','Land Rover.':'Jaguar','Fiat':'Chrysler',
                                                           'Astra':'Opel'
                                                          ,'Seat':'Volkswagen','Lincoln':'Ford','LIBERTY':'Jeep','Wrangler Jeep':'Jeep'
                                                           ,'Buik':'GMC','COMMANDER':'Jeep','Datsun':'Nissan','.':'Unknown'})
test.Subject_Car_Make.fillna('Unknown',inplace=True)

In [40]:
train.isnull().sum()/len(train)* 100

Gender                            0.0
Age                               0.0
No_Pol                            0.0
Car_Category                      0.0
Subject_Car_Colour                0.0
Subject_Car_Make                  0.0
LGA_Name                          0.0
State                             0.0
ProductName                       0.0
Policy Start Date_year            0.0
Policy Start Date_day             0.0
Policy Start Date_month           0.0
Policy Start Date_quarter         0.0
Policy End Date_year              0.0
Policy End Date_day               0.0
Policy End Date_month             0.0
Policy End Date_quarter           0.0
First Transaction Date_year       0.0
First Transaction Date_day        0.0
First Transaction Date_month      0.0
First Transaction Date_quarter    0.0
dtype: float64

In [41]:
from sklearn.preprocessing import LabelEncoder
le =LabelEncoder()
train['Subject_Car_Make']=le.fit_transform(train['Subject_Car_Make'])
train['LGA_Name']=le.fit_transform(train['LGA_Name'])
train['State']=le.fit_transform(train['State'])
train['ProductName']=le.fit_transform(train['ProductName'])
train['Gender']=le.fit_transform(train['Gender'])
train['No_Pol']=le.fit_transform(train['No_Pol'])
train['Subject_Car_Colour']=le.fit_transform(train['Subject_Car_Colour'])
train['Car_Category']=le.fit_transform(train['Car_Category'])

In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12079 entries, ID_0040R73 to ID_ZZWRIIE
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype
---  ------                          --------------  -----
 0   Gender                          12079 non-null  int64
 1   Age                             12079 non-null  int64
 2   No_Pol                          12079 non-null  int64
 3   Car_Category                    12079 non-null  int64
 4   Subject_Car_Colour              12079 non-null  int64
 5   Subject_Car_Make                12079 non-null  int64
 6   LGA_Name                        12079 non-null  int64
 7   State                           12079 non-null  int64
 8   ProductName                     12079 non-null  int64
 9   Policy Start Date_year          12079 non-null  int64
 10  Policy Start Date_day           12079 non-null  int64
 11  Policy Start Date_month         12079 non-null  int64
 12  Policy Start Date_quarter       12079 non-null  int

In [43]:
test['Subject_Car_Make']=le.fit_transform(test['Subject_Car_Make'])
test['LGA_Name']=le.fit_transform(test['LGA_Name'])
test['State']=le.fit_transform(test['State'])
test['ProductName']=le.fit_transform(test['ProductName'])
test['Gender']=le.fit_transform(test['Gender'])
test['No_Pol']=le.fit_transform(test['No_Pol'])
test['Subject_Car_Colour']=le.fit_transform(test['Subject_Car_Colour'])
test['Car_Category']=le.fit_transform(test['Car_Category'])

In [44]:
test.shape

(5177, 21)

In [37]:
test.info()


<class 'pandas.core.frame.DataFrame'>
Index: 5177 entries, ID_009D84L to ID_ZZYTLV1
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype
---  ------                          --------------  -----
 0   Gender                          5177 non-null   int32
 1   Age                             5177 non-null   int64
 2   No_Pol                          5177 non-null   int64
 3   Car_Category                    5177 non-null   int32
 4   Subject_Car_Colour              5177 non-null   int32
 5   Subject_Car_Make                5177 non-null   int32
 6   LGA_Name                        5177 non-null   int32
 7   State                           5177 non-null   int32
 8   ProductName                     5177 non-null   int32
 9   Policy Start Date_year          5177 non-null   int64
 10  Policy Start Date_day           5177 non-null   int64
 11  Policy Start Date_month         5177 non-null   int64
 12  Policy Start Date_quarter       5177 non-null   int6

In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12079 entries, ID_0040R73 to ID_ZZWRIIE
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype
---  ------                          --------------  -----
 0   Gender                          12079 non-null  int64
 1   Age                             12079 non-null  int64
 2   No_Pol                          12079 non-null  int64
 3   Car_Category                    12079 non-null  int64
 4   Subject_Car_Colour              12079 non-null  int64
 5   Subject_Car_Make                12079 non-null  int64
 6   LGA_Name                        12079 non-null  int64
 7   State                           12079 non-null  int64
 8   ProductName                     12079 non-null  int64
 9   Policy Start Date_year          12079 non-null  int64
 10  Policy Start Date_day           12079 non-null  int64
 11  Policy Start Date_month         12079 non-null  int64
 12  Policy Start Date_quarter       12079 non-null  int

In [46]:
X = train


In [49]:
 test_new=test

In [50]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12079 entries, ID_0040R73 to ID_ZZWRIIE
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype
---  ------                          --------------  -----
 0   Gender                          12079 non-null  int64
 1   Age                             12079 non-null  int64
 2   No_Pol                          12079 non-null  int64
 3   Car_Category                    12079 non-null  int64
 4   Subject_Car_Colour              12079 non-null  int64
 5   Subject_Car_Make                12079 non-null  int64
 6   LGA_Name                        12079 non-null  int64
 7   State                           12079 non-null  int64
 8   ProductName                     12079 non-null  int64
 9   Policy Start Date_year          12079 non-null  int64
 10  Policy Start Date_day           12079 non-null  int64
 11  Policy Start Date_month         12079 non-null  int64
 12  Policy Start Date_quarter       12079 non-null  int

In [51]:
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

In [51]:
test.info()

AttributeError: 'numpy.ndarray' object has no attribute 'info'

In [1]:
X.shape

NameError: name 'X' is not defined

In [2]:
test.shape

NameError: name 'test' is not defined

In [52]:
seed = 34890
skf = StratifiedKFold(n_splits=25, shuffle=True, random_state=seed)
scores = []
preds = []
rforest = RandomForestClassifier(n_jobs=-1,random_state =seed, n_estimators=500)
#creating a for loop for the stratified k fold
i = 1
for train, test in skf.split(X, y):
    x_train, x_test, y_train, y_test = X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test]
    rforest.fit(x_train, y_train)
    print('Number of splits trained {} '.format(i))
    score = f1_score(y_test, rforest.predict(x_test))
    pred = rforest.predict_proba(test_new)[:, 1]
    scores.append(score)
    preds.append(pred)
    i += 1

np.mean(scores)

Number of splits trained 1 
Number of splits trained 2 
Number of splits trained 3 
Number of splits trained 4 
Number of splits trained 5 
Number of splits trained 6 
Number of splits trained 7 
Number of splits trained 8 
Number of splits trained 9 
Number of splits trained 10 
Number of splits trained 11 
Number of splits trained 12 
Number of splits trained 13 
Number of splits trained 14 
Number of splits trained 15 
Number of splits trained 16 
Number of splits trained 17 
Number of splits trained 18 
Number of splits trained 19 
Number of splits trained 20 
Number of splits trained 21 
Number of splits trained 22 
Number of splits trained 23 
Number of splits trained 24 
Number of splits trained 25 


0.1786996703062525

In [57]:
preds_mean = np.mean(preds, axis=0)

final = []
for x in preds_mean:
    if x > 0.3:
        final.append(1)
    else:
        final.append(0)
        
submit = ss.copy()
submit.target = final
submit.to_csv('Baseline.csv',index=False)